In [9]:
# Accident_bicycle

In [7]:
import json
geo_path = 'bicycle_Seoul.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '서울특별시 강동구1',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.1357936, 37.530832],
      [127.1354508, 37.5310561],
      [127.1350596, 37.5312226],
      [127.1346351, 37.5313251],
      [127.1341937, 37.5313598],
      [127.1337522, 37.5313251],
      [127.1333277, 37.5312226],
      [127.1329365, 37.5310561],
      [127.1325937, 37.530832],
      [127.1323123, 37.5305589],
      [127.1321032, 37.5302473],
      [127.1319744, 37.5299093],
      [127.131931, 37.5295577],
      [127.1319745, 37.5292062],
      [127.1321032, 37.5288682],
      [127.1323123, 37.5285566],
      [127.1325937, 37.5282835],
      [127.1329366, 37.5280595],
      [127.1333278, 37.5278929],
      [127.1337522, 37.5277904],
      [127.1341937, 37.5277558],
      [127.1346351, 37.5277904],
      [127.1350595, 37.5278929],
      [127.1

In [8]:
# 데이터 정리 
import numpy as np
import pandas as pd
bicycle = pd.read_csv('Accident_bicycle.csv', encoding='euc-kr')

bicycle = bicycle[['시도시군구명', '발생건수', '경도', '위도']]
bicycle = bicycle.rename(columns={'시도시군구명':'구별'})

bicycle = bicycle[0:187]
bicycle = bicycle[['구별', '발생건수', '경도', '위도']]

bicycle.set_index('구별', inplace=True)

import folium
import pandas as pd

map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                tiles='openstreetmap')
# tiles='cartodbpositron'
soo = folium.Choropleth(geo_data=geo_str,
               data = bicycle['발생건수'],
               columns = [bicycle.index, bicycle['발생건수']],
               fill_color='PuRd',
               key_on='feature.properties.name',
               highlight=True,
               fill_opacity=0.5, line_opacity=1).add_to(map)

folium.GeoJsonTooltip(fields = ['name']).add_to(soo.geojson)
map

In [9]:
import os
import folium
import pandas as pd
import numpy as np
import networkx as nx

nodes = pd.read_csv('Seoul_nodes_190709.csv')
links = pd.read_csv('Seoul_links_190709.csv')

nodes = nodes[['Id','NODE_NAME','latitude','longitude']]

links = links[['Source','Target']]
links.head()

source_in = links['Source'].apply(lambda x : x in list(nodes['Id'])) # check Sources are in incheon_id
target_in = links['Target'].apply(lambda x : x in list(nodes['Id'])) # check Targets are in incheon_id
# source_in and target_in are boolean type pandas.Series which contains True or False
Seoul_links = links[source_in & target_in] # contain if both target and source are contained in incheon_id

# Positioning the Standard Point for our Folium Map
std_point = tuple(nodes.head(1)[['latitude','longitude']].iloc[0])
std_point

map_osm = folium.Map(location=std_point, zoom_start=10) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

kw = {'opacity': 0.5, 'weight': 2}
for ix, row in Seoul_links.iterrows():
    start = tuple(nodes[nodes['Id']==row['Source']][['latitude','longitude']].iloc[0])
    end = tuple(nodes[nodes['Id']==row['Target']][['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        **kw,
    ).add_to(map_osm)
    
soo = folium.Choropleth(geo_data=geo_str,
                   data = bicycle['발생건수'],
                   columns = [bicycle.index, bicycle['발생건수']],
                   fill_color='PuRd',
                   key_on='feature.properties.name',
                   highlight=True,
                   fill_opacity=0.5, line_opacity=1).add_to(map_osm)

folium.GeoJsonTooltip(fields = ['name']).add_to(soo.geojson)
    
map_osm.save('Accident_bicycle_road.html')